# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title' 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [8]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [9]:
dtm

<67513x24939 sparse matrix of type '<class 'numpy.int64'>'
	with 350924 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [12]:
len(cv.get_feature_names_out())

24939

In [13]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'или', 'российские', 'украинцы'],
      dtype=object)

In [14]:
len(lda.components_)

7

In [15]:
lda.components_

array([[1.43076778e-01, 1.43237869e-01, 1.42857263e-01, ...,
        1.42857595e-01, 1.42857335e-01, 1.42857531e-01],
       [1.42857260e-01, 2.44916301e+02, 1.43361767e-01, ...,
        1.42857659e-01, 1.42857363e-01, 1.42857587e-01],
       [1.43203007e-01, 5.46307097e+01, 1.42857287e-01, ...,
        1.14298123e+00, 2.14285584e+00, 1.42857620e-01],
       ...,
       [3.14225166e+00, 3.90974459e-01, 1.42857293e-01, ...,
        1.42857721e-01, 1.42857389e-01, 1.42857640e-01],
       [1.42857230e-01, 3.63394956e+01, 1.42857242e-01, ...,
        1.42857528e-01, 1.42857307e-01, 1.42857474e-01],
       [1.42896795e-01, 9.57316137e+01, 2.14235186e+00, ...,
        1.14273057e+00, 1.42857387e-01, 2.14285453e+00]])

In [16]:
len(lda.components_[0])

24939

### Show most important words of first extracted topic

In [17]:
first_topic = lda.components_[0]

In [18]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([22881,  5717, 12715, ..., 21221, 21919,  2841], dtype=int64)

In [19]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [20]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

afd
wahrheit
bundesliga
bayern
wahl
ex
cdu
trump
us
berlin


### Show most important words of first extracted topic

In [21]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['union', 'letzte', 'generation', 'spd', 'gericht', 'afd', 'wahrheit', 'bundesliga', 'bayern', 'wahl', 'ex', 'cdu', 'trump', 'us', 'berlin']


Die TOP-15 Wörter für das Thema #1
['biden', 'millionen', 'lindner', 'prozent', '000', 'neue', 'weg', 'gas', 'inflation', 'deutsche', 'deutschland', 'us', 'iran', 'regierung', 'menschen']


Die TOP-15 Wörter für das Thema #2
['lauterbach', 'afghanistan', 'macron', 'frauen', 'usa', 'steingart', 'gabor', 'dfb', 'tote', 'gastbeitrag', 'polen', 'eu', 'wm', 'frankreich', 'deutschland']


Die TOP-15 Wörter für das Thema #3
['habeck', 'analyse', 'warnt', 'ampel', 'news', 'chinas', 'afd', 'kritik', 'taiwan', 'lage', 'marktbericht', 'pandemie', 'invasion', 'china', 'corona']


Die TOP-15 Wörter für das Thema #4
['spd', 'ex', 'tot', 'scholz', 'wegen', 'grünen', 'ampel', 'reitz', 'analyse', 'ulrich', 'stream', 'bundestag', 'türkei', 'streit', 'nord']


Die TOP-15 Wörter für das Thema #5
['nachrichten', 'usa', 'russen', 'r

### Get topics with highest probability for news items in train data

In [22]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [23]:
topic_results[0].round(5)

array([0.01791, 0.01792, 0.01786, 0.89272, 0.01786, 0.01786, 0.01786])

In [24]:
topic_results[0].argmax()

3

In [25]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [26]:
train.head()

,date,title,description,category,text,source,title_description_text,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,3
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,5
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,4
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,5
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,4


## Visualize topics

In [27]:
# Generate a word cloud image for given topic
#def draw_word_cloud(index):
#    imp_words_topic=""
#    comp=lda.components_[index]
#    vocab_comp = zip(vocab, comp)
#    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:50]
#    for word in sorted_words:
#        imp_words_topic=imp_words_topic+" "+word[0]

#    wordcloud = WordCloud(width=600, height=400).generate(imp_words_topic)
#    plt.figure( figsize=(5,5))
#    plt.imshow(wordcloud)
#    plt.axis("off")
#    plt.tight_layout()
#    plt.show()
    
#for i in range(6):
#    draw_word_cloud(i)

## Save train set with topics and dump model

In [28]:
#Save result to csv
train.to_csv('train_lda_topics_7.csv')

In [29]:
# Dump LDA model
joblib.dump(lda, 'lda_model_7_topics.jl')

['lda_model_7_topics.jl']